<img src="mioti.png" style="height: 100px">
<center style="color:#888">Data Science with Python</center>

# DSPy2. NumPy "advanced". Challenge

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Broadcasting-into-practice:-feature-scaling-through-standarization" data-toc-modified-id="Broadcasting-into-practice:-feature-scaling-through-standarization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Broadcasting into practice: feature scaling through standarization</a></span></li><li><span><a href="#Sistemas-de-coordenadas---Vectorización" data-toc-modified-id="Sistemas-de-coordenadas---Vectorización-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sistemas de coordenadas - Vectorización</a></span></li></ul></div>

## Broadcasting into practice: feature scaling through standarization

Con algunas excepciones, los algoritmos de Machine Learning no funcionan del todo bien si los atributos numéricos que se les pasan tienen escalas muy diferentes ([further reading](http://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html)). Una técnica que puede usarse para que todos los atributos tengan la misma escala es la estandarización: sustraerles la media (para que sea 0) y dividirlos por la desviación típica (de modo que la varianza resultante sea 1). 

a) Imaginemos que tenemos un array con datos de un sensor en una playa, que mide cuatro atributos como serían calidad del agua, viento, oleaje y temperatura, en distintas unidades que tienen distintas escalas. Esas mediciones nos llegan en un array con 3 columnas (una por feature) y 1000 filas (una fila por cada instante de tiempo del que tenemos las medidas). 

In [2]:
import numpy as np
from numpy import newaxis
data_0 = np.random.normal(loc=[-10, 0, 10], scale=[2, 1, 2], size=(1000,3))
#calculamos la media por columna y dividimos por la desviacion tipica
result = (data_0-data_0.mean(axis=0))/data_0.std(axis=0)
result

array([[ 0.43393177, -0.49039814, -0.09412555],
       [ 1.64234245,  0.76850845,  0.99330882],
       [ 0.7022275 , -2.34639893, -1.15703004],
       ...,
       [-0.75078629, -0.05489232,  0.14466989],
       [ 1.17976677, -1.06610893, -0.44125851],
       [-0.34509435, -0.12436662, -0.08498875]])

Estandariza esa matriz de manera vectorial: réstale la media por columna y divide por la desviación típica de cada columna.

b) Imaginemos ahora que el array 2d tiene una fila por cada feature y una columna por cada medición (la transpuesta de la matriz anterior). ¿Cómo vectorizamos la operación ahora? (réstale la media por fila y divide por la desviación típica por fila). ¡No vale transponer data!

In [235]:
# usammos los mismos datos para comparar luego el resultado
data_1 = data_0.T
# Expandimos la dimension usando la funcion newaxis de numpy que nos permite incrementar la dimension de una array en 1,
# en este caso axis=1, es decir [:,newaxis] y al mismo tiempo efecutamos el calculo de restar la media por fila y dividir
# por la desviacion tipica
result2 = (data_1 - data_1.mean(axis=1)[:,newaxis]) / data_1.std(axis=1)[:,newaxis] 
result2

array([[ 0.54769695,  0.48526373, -2.04897274, ..., -1.07914602,
         0.0234247 , -0.86733924],
       [ 0.08933853, -0.59543409, -0.54169611, ..., -0.7242368 ,
         3.05247208,  0.94807417],
       [ 0.65321363, -0.10818826, -0.20049989, ...,  2.06379985,
        -2.72702526, -1.39290317]])

In [233]:
%timeit result2

17.6 ns ± 0.218 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [224]:
# comparamos el resultado restando a la matriz result del primer apartado la matriz traspuesta del segundo apartado.
result - result2.T

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

## Sistemas de coordenadas - Vectorización

Vamos a escribir funciones para convertir puntos dados en un sistema de coordenadas cartesianas a los mismos puntos expresados en un sistema de coordenadas polares.

<img src="coordinates.png" style="height: 200px;float: left">

Dado un punto en coordenadas cartesianas $(x,y)$, sus correspondientes coordenadas polares $(r, \varphi)$ se obtienen como sigue:

\begin{equation}
r = \sqrt{(x^2+y^2)}
\end{equation}
\begin{equation}
\varphi = arctan\left(\frac{y}{x}\right)
\end{equation}

<div style="clear: left"/>

* Escribe una función usando NumPy y otra basada en Python puro tal que, dado un array o lista, respectivamente, de coordenadas cartesianas, devuelva un array o lista de coordenadas polares. 
* Compara su rendimiento (hint: magic commands de Jupyter notebook)



In [5]:
num_muestras_input = 10000
Z_cartesian = np.random.random((num_muestras_input,2))

In [6]:
import numpy as np 
def coordenadas_polares(array):
    # Calculamos las coordenadas polares usando la formula de arriba 
    return np.concatenate((np.sqrt(array[0:,:1]*2 + array[0:,1:]*2),np.arctan2(array[0:,:1],array[0:,1:])),axis=1)
coordenadas_polares(Z_cartesian)

array([[1.60851605, 0.69099194],
       [1.67188526, 0.76099927],
       [1.5409042 , 0.60587197],
       ...,
       [1.32165291, 0.00532324],
       [1.07611976, 1.050646  ],
       [1.20074163, 0.4592738 ]])

In [9]:
%timeit coordenadas_polares(Z_cartesian)


379 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
import math
def coordenadas_polares_python(array):
    res = []
    for i in array:
        res.append([math.sqrt(i[0]*2 + math.sqrt(i[1]*2)),math.atan2(i[1],i[0])])
    return res
res_coordenadas_polares = coordenadas_polares_python(Z_cartesian)
res_coordenadas_polares

[[1.536616911965307, 0.8798043875996222],
 [1.60001031948286, 0.8097970527350259],
 [1.4683576029708443, 0.9649243597238983],
 [1.621408839303979, 0.7331476281331831],
 [1.3676981975879288, 0.7024568140630503],
 [1.3999073317814652, 0.5383873364008243],
 [0.9632320243589191, 0.25110988280660623],
 [1.70790014652204, 0.7147050950724242],
 [1.5648259594950908, 0.9090023203105888],
 [1.5179724495210545, 0.3062814500063675],
 [1.1244644783591378, 0.7281110833074256],
 [1.745504145965016, 0.6527264253231632],
 [1.2565049934960948, 0.09750803637325482],
 [1.425683529160563, 0.3068794532170711],
 [1.0825579119268889, 0.32373843477024433],
 [1.290586840113964, 1.2088763261429414],
 [1.670998483510632, 0.7303260782955537],
 [1.7747368488490236, 0.7287811180427191],
 [1.510574337309876, 0.527604201881186],
 [1.6001113308498196, 0.3451591411053405],
 [1.1607832888380578, 0.9452124210540204],
 [1.6525679744276427, 0.6319100812755001],
 [1.4436159599629228, 1.2127133474816298],
 [1.4475049843866485

In [11]:
%timeit res_coordenadas_polares

20.3 ns ± 0.528 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
